Connected to spyder (Python 3.12.3)

In [1]:
import xarray as xr
import geopandas as gpd
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import rioxarray as rxr
import numpy as np
import rasterio
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
lithology = gpd.read_file(r"C:\Users\gmfet\Desktop\predictors\lithology\litology_italy.gpkg")
output_path = r'C:\Users\gmfet\Desktop\collaboration_with_Awais\lithology.tif'

# Get the bounds of the file
xmin, ymin, xmax, ymax = lithology.total_bounds
res = 0.001  # pixel size

# width = int((xmax - xmin) / res)
height = int((ymax - ymin) / res)
width = height
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

shapes = ((geom, value) for geom, value in zip(lithology.geometry, lithology["cat"]))

with rasterio.open(output_path, 'w', driver='GTiff', width=width, height=height, count=1, crs=lithology.crs, transform=transform, dtype='uint8') as geo_pckg:
    geo_pckg.write(rasterize(shapes, out_shape=(height, width), fill=0, transform=transform, dtype='uint8'), 1)


In [2]:
# italy shapefile 
shp_path = r"C:\Users\gmfet\vgd_italy\italy_aoi\gadm41_ITA_0.shp"

shp_file = gpd.read_file(shp_path)

shp_file.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Static features
Save the categorical variables as uint8 dtype


In [ ]:
import os
import xarray as xr


filepath = r'C:\Users\gmfet\vgd_italy\regression\original_data'

file_list = os.listdir(filepath)
for file in file_list:
    if file.endswith('.tif'):
        engine='rasterio'
    elif file.endswith('.nc'):
        engine = 'h5netcdf'
    else:
        continue
    
    print(file)
    
    with xr.open_dataset(os.path.join(filepath, file), engine=engine) as data_ds:
        rename_dict = {'x': 'longitude', 'y': 'latitude', 'lon': 'longitude', 'lat': 'latitude', 'band':'time'}
        data_ds = data_ds.rename({k: v for k, v in rename_dict.items() if k in data_ds.dims})
        print(data_ds.dims)

In [24]:
# with rxr.open_rasterio(r"C:\Users\gmfet\Desktop\predictors\LULC\*.tif", chunks=1000) as ds:
#     ds
with xr.open_mfdataset(r"C:\Users\gmfet\vgd_italy\regression\original_data\ksat.nc", engine='netcdf4') as ds:
    print(ds)
    ds = ds.rio.write_crs("EPSG:4326", inplace=False)


    # ds['band_data'] = ds['band_data'].rio.write_nodata(np.nan)
    # ds = ds.astype('float32')
    # ds = ds.rio.clip(shp_file.geometry, shp_file.crs, drop=True, all_touched=True)
    
    data_attrs = ds.attrs.copy()
    coord_attrs = {c: ds[c].attrs.copy() for c in ds.coords}
    # ds = ds.squeeze("band", drop=True)
    # # # ds = ds.drop_vars("spatial_ref")
    ds = ds.rename({"lon": "longitude", "lat": "latitude", 'Band1': 'ksat'})
    ds = ds.chunk({'latitude': 256, 'longitude': 256})

    


    # ds = ds.rename({"x": "longitude", "y": "latitude", 'band': 'time', 'band_data': 'drought_code'})
    # ds['time'] = pd.date_range('2017-01-01', periods=ds.sizes['time'], freq='D')
    # ds = ds.chunk({'time': -1})
    # ds = ds.interpolate_na(
    #     dim="time", 
    #     method="polynomial", 
    #     order=3,
    #     fill_value="extrapolate"
        
    # )

    # ds = ds.chunk({'time': -1, 'latitude': 256, 'longitude': 256})

    # # # # # # ds = ds.to_dataset(name='clay_content')
    ds.attrs.update(data_attrs)
    ds = ds.astype("float32")

    # restore coordinate attributes
    for c in ["longitude", "latitude"]:
        if c in coord_attrs:
            ds[c].attrs.update(coord_attrs[c])

    # # # ds = ds.dropna('latitude')

    # # # ds = ds.rio.clip(shp_file.geometry, shp_file.crs, drop=True, all_touched=True)
            
    ds.to_netcdf("ksat.nc", engine="h5netcdf", encoding= {'ksat': {"dtype": "float32", "zlib": True, "complevel": 0, "compression": "gzip", "compression_opts": 0}})
    

<xarray.Dataset> Size: 104MB
Dimensions:  (lon: 7200, lat: 3600)
Coordinates:
  * lon      (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float64 29kB 89.97 89.92 89.88 89.82 ... -89.88 -89.92 -89.97
Data variables:
    crs      int32 4B ...
    Band1    (lat, lon) float32 104MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
Attributes:
    CDI:                        Climate Data Interface version 1.9.8 (https:/...
    Conventions:                CF-1.5
    history:                    Wed Dec 16 16:42:50 2020: cdo -mulc,10. ksat3...
    GDAL_AREA_OR_POINT:         Area
    GDAL:                       GDAL 3.0.4, released 2020/01/28
    history_of_appended_files:  Fri Dec 11 16:53:00 2020: Appended file /huge...
    NCO:                        netCDF Operators version 4.9.2 (Homepage = ht...
    CDO:                        Climate Data Operators version 1.9.8 (https:/...


## Dynamic features

In [ ]:
# with rxr.open_rasterio(r"C:\Users\gmfet\Desktop\predictors\LULC\*.tif", chunks=1000) as ds:
#     ds
# from dask.distributed import Client
# client = Client() # This will give you a link to a dashboard (usually http://localhost:8787)

encoding = {
    "temperature": {
        "dtype": "float32",        # Store as 16-bit integers
        "scale_factor": 0.01,   # Preservation of 2 decimal places
        # "_FillValue": -9999,    # Map NaNs to -9999
        "zlib": True,           # Enable compression
        "complevel": 0,         # Moderate compression level
        # "shuffle": True,        # Better compression efficiency
        # "chunksizes": (1, 180, 360) # Optimize for time-series access
    }
}

# ds.to_netcdf("output.nc", encoding=encoding)
with xr.open_mfdataset(r"C:\Users\gmfet\vgd_italy\data\dynamic\temperature.tif", engine='rasterio') as ds:
    ds
    ds['band_data'] = ds['band_data'].rio.write_nodata(np.nan)
    # ds = ds.chunk({'x': 1024, 'y': 1024})
    ds = ds.astype('float32')
    # # # # # ds = ds.rio.clip(shp_file.geometry, shp_file.crs, drop=True, all_touched=True)
    
    # ds = ds.fillna(0).astype("float32")
    data_attrs = ds.attrs.copy()
    coord_attrs = {c: ds[c].attrs.copy() for c in ds.coords}
    # # # ds = ds.drop_vars("spatial_ref")
    ds = ds.rename({"x": "longitude", "y": "latitude", 'band': 'time', 'band_data': 'drought_code'})
    ds['time'] = pd.date_range('2017-01-01', periods=ds.sizes['time'], freq='D')
    # ds = ds.chunk({'longitude': -1, 'latitude': 1024, 'time': 512})

    # ds = ds.interpolate_na(
    #     dim="longitude", 
    #     method="linear", 
    #     fill_value="extrapolate"
        
    # )

    ds = ds.chunk({'time': -1, 'latitude': 1024, 'longitude': 1024,})

    # ds = ds.interpolate_na(
    #     dim="time", 
    #     method="linear", 
    #     fill_value="extrapolate"
        
    # )

    
    

    # # # # # # # ds = ds.to_dataset(name='bulk_density')
    ds.attrs.update(data_attrs)
    ds = ds.astype("float32")



    

    # restore coordinate attributes
    for c in ["longitude", "latitude"]:
        if c in coord_attrs:
            ds[c].attrs.update(coord_attrs[c])

    # # # # ds = ds.dropna('latitude')

    # # # # ds = ds.rio.clip(shp_file.geometry, shp_file.crs, drop=True, all_touched=True)
    
    ds.to_netcdf("temperature.nc", engine="h5netcdf", encoding= encoding)
    

In [ ]:
# ds = ds.astype("uint8")
print(ds)

In [ ]:
ds['temperature'].sel(time='2018-08-01').plot()
# plt.show()

In [ ]:

np.unique(ds['drought_code'].sel(time='2017-01-01').values)